In [1]:
import pandas as pd
import numpy as np
#import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

# Unindo a Base de Dados com os Jogos do Dia

In [2]:
#back draw, 2.5
df1 = pd.read_excel("G:\\Meu Drive\\Programacao\\futPythonTrader\\BasesDeDados\\x_FutPythonTrader_Base_de_Dados_Temporada_2122_x.xlsx")
df2 = pd.read_excel("https://github.com/futpythontrader/YouTube/raw/main/x_FutPythonTrader_Base_de_Dados_Temporada_Atual_x.xlsx")
base = pd.concat([df1,df2])
base.reset_index(inplace=True, drop=True)
base.index = base.index.set_names(['Nº'])
base = base.rename(index=lambda x: x + 1)

base = base[['Date','League','Home','Away','Season','FT_Goals_H','FT_Goals_A','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Odds_Over25','FT_Odds_Under25','Odds_BTTS_Yes','Odds_BTTS_No']]
base["Date"] = pd.to_datetime(base["Date"])


In [4]:
baseBackDraw = base[base['League'].isin(["Austria Bundesliga",
"Bolivia LFPB",
"Brazil Serie B",
"Brazil Serie D",
"Canada Canadian Premier League",
"China Chinese Super League",
"Croatia Prva HNL",
"Czech Republic First League",
"Denmark Superliga",
"Germany 2. Bundesliga",
"Italy Serie A",
"Italy Serie B",
"Mexico Liga MX",
"Poland Ekstraklasa",
"Romania Liga I",
"Scotland Premiership",
"Spain La Liga",
"Switzerland Super League",
"Uruguay Primera División",
"USA MLS",
"Venezuela Primera División",
"Wales Welsh Premier League"]) == True]

In [5]:
flt = ((baseBackDraw.FT_Odds_H != 0) & (baseBackDraw.FT_Odds_D != 0) & (baseBackDraw.FT_Odds_A != 0))
baseBackDraw = baseBackDraw[flt]

baseBackDraw["Total_Goals"] = baseBackDraw["FT_Goals_H"] + baseBackDraw["FT_Goals_A"]

baseBackDraw.loc[baseBackDraw['FT_Goals_H'] > baseBackDraw['FT_Goals_A'], 'FT_Result'] = "H"
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] == baseBackDraw['FT_Goals_A'], 'FT_Result'] = "D"
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] < baseBackDraw['FT_Goals_A'], 'FT_Result'] = "A"

# Profit
green = baseBackDraw.FT_Odds_D - 1
red = -1

baseBackDraw.loc[baseBackDraw['FT_Result'] == "D", 'Profit'] = green
baseBackDraw.loc[baseBackDraw['FT_Result'] != "D", 'Profit'] = red

# Probabilidades
baseBackDraw['p_H'] = 1 / baseBackDraw.FT_Odds_H
baseBackDraw['p_D'] = 1 / baseBackDraw.FT_Odds_D
baseBackDraw['p_A'] = 1 / baseBackDraw.FT_Odds_A

# CV das Odds do Match Odds
baseBackDraw['CV_HDA'] = baseBackDraw[['p_H','p_D','p_A']].std(ddof=0, axis=1) / baseBackDraw[['p_H','p_D','p_A']].mean(axis=1)

# Pontos
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] > baseBackDraw['FT_Goals_A'], 'Ptos_H'] = 3
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] == baseBackDraw['FT_Goals_A'], 'Ptos_H'] = 1
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] < baseBackDraw['FT_Goals_A'], 'Ptos_H'] = 0

baseBackDraw.loc[baseBackDraw['FT_Goals_H'] < baseBackDraw['FT_Goals_A'], 'Ptos_A'] = 3
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] == baseBackDraw['FT_Goals_A'], 'Ptos_A'] = 1
baseBackDraw.loc[baseBackDraw['FT_Goals_H'] > baseBackDraw['FT_Goals_A'], 'Ptos_A'] = 0

baseBackDraw["Ptos_H"] = baseBackDraw["Ptos_H"].astype('int64')
baseBackDraw["Ptos_A"] = baseBackDraw["Ptos_A"].astype('int64')

baseBackDraw['Media_Ptos_H'] = baseBackDraw.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
baseBackDraw['Media_Ptos_A'] = baseBackDraw.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

baseBackDraw['Media_Ptos_H'] = baseBackDraw.groupby('Home')['Media_Ptos_H'].shift(1)
baseBackDraw['Media_Ptos_A'] = baseBackDraw.groupby('Away')['Media_Ptos_A'].shift(1)

baseBackDraw['DesvPad_Ptos_H'] = baseBackDraw.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
baseBackDraw['DesvPad_Ptos_A'] = baseBackDraw.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

baseBackDraw['DesvPad_Ptos_H'] = baseBackDraw.groupby('Home')['DesvPad_Ptos_H'].shift(1)
baseBackDraw['DesvPad_Ptos_A'] = baseBackDraw.groupby('Away')['DesvPad_Ptos_A'].shift(1)

baseBackDraw['CV_Ptos_H'] = baseBackDraw['DesvPad_Ptos_H'] / baseBackDraw['Media_Ptos_H']
baseBackDraw['CV_Ptos_A'] = baseBackDraw['DesvPad_Ptos_A'] / baseBackDraw['Media_Ptos_A']

# Valor do Gol
baseBackDraw['VG_H'] = baseBackDraw.FT_Goals_H * baseBackDraw.p_A
baseBackDraw['VG_A'] = baseBackDraw.FT_Goals_A * baseBackDraw.p_H

baseBackDraw['Media_VG_H'] = baseBackDraw.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
baseBackDraw['Media_VG_A'] = baseBackDraw.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

baseBackDraw['Media_VG_H'] = baseBackDraw.groupby('Home')['Media_VG_H'].shift(1)
baseBackDraw['Media_VG_A'] = baseBackDraw.groupby('Away')['Media_VG_A'].shift(1)

baseBackDraw['DesvPad_VG_H'] = baseBackDraw.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
baseBackDraw['DesvPad_VG_A'] = baseBackDraw.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

baseBackDraw['DesvPad_VG_H'] = baseBackDraw.groupby('Home')['DesvPad_VG_H'].shift(1)
baseBackDraw['DesvPad_VG_A'] = baseBackDraw.groupby('Away')['DesvPad_VG_A'].shift(1)

baseBackDraw['CV_VG_H'] = baseBackDraw['DesvPad_VG_H'] / baseBackDraw['Media_VG_H']
baseBackDraw['CV_VG_A'] = baseBackDraw['DesvPad_VG_A'] / baseBackDraw['Media_VG_A']

# Custo do Gol
baseBackDraw['CG_H'] = baseBackDraw.p_H / baseBackDraw.FT_Goals_H
baseBackDraw['CG_A'] = baseBackDraw.p_A / baseBackDraw.FT_Goals_A

baseBackDraw['Media_CG_H'] = baseBackDraw.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
baseBackDraw['Media_CG_A'] = baseBackDraw.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

baseBackDraw['Media_CG_H'] = baseBackDraw.groupby('Home')['Media_CG_H'].shift(1)
baseBackDraw['Media_CG_A'] = baseBackDraw.groupby('Away')['Media_CG_A'].shift(1)

baseBackDraw['DesvPad_CG_H'] = baseBackDraw.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
baseBackDraw['DesvPad_CG_A'] = baseBackDraw.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

baseBackDraw['DesvPad_CG_H'] = baseBackDraw.groupby('Home')['DesvPad_CG_H'].shift(1)
baseBackDraw['DesvPad_CG_A'] = baseBackDraw.groupby('Away')['DesvPad_CG_A'].shift(1)

baseBackDraw['CV_CG_H'] = baseBackDraw['DesvPad_CG_H'] / baseBackDraw['Media_CG_H']
baseBackDraw['CV_CG_A'] = baseBackDraw['DesvPad_CG_A'] / baseBackDraw['Media_CG_A']

baseBackDraw.replace(np.inf, 1, inplace=True)
baseBackDraw = drop_reset_index(baseBackDraw)

baseBackDraw = baseBackDraw.drop(['Ptos_H','Ptos_A','VG_H', 'VG_A', 'CG_H', 'CG_A'], axis=1)

In [6]:
base = drop_reset_index(base)
#base

In [7]:
base_HBackDraw = baseBackDraw[['Home', 'Media_Ptos_H', 'DesvPad_Ptos_H', 'CV_Ptos_H', 'Media_VG_H', 'DesvPad_VG_H', 'CV_VG_H', 'Media_CG_H', 'DesvPad_CG_H', 'CV_CG_H']]

base_HBackDraw = baseBackDraw[['Home', 'CV_HDA', 'Media_VG_H', 'CV_CG_H']]

base_ABackDraw = baseBackDraw[['Away', 'CV_HDA', 'Media_VG_A', 'CV_CG_A']]

#base_H.to_excel('baseh.xlsx', index=None)

In [8]:
from datetime import date, datetime, timedelta
dia = date.today() - timedelta(-1)
print(dia)

2023-07-09


In [9]:
#jogos_do_dia = pd.read_csv('https://github.com/futpythontrader/YouTube/blob/main/Jogos_do_Dia_FlashScore/'+str(dia)+'_Jogos_do_Dia_FlashScore.csv?raw=true')
jogos_do_dia = pd.read_excel('G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\'+str(dia)+'_Jogos_do_Dia_FlashScore.xlsx')
#jogos_do_dia = pd.read_excel('G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\2023-04-22_Jogos_do_Dia_FlashScore.xlsx')
#jogos_do_dia = jogos_do_dia[['League','Round','Date','Time','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']] 
jogos_do_dia = jogos_do_dia[['League','Date','Time','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']] 
jogos_do_dia.dropna(inplace=True)
jogos_do_dia.reset_index(inplace=True, drop=True)
jogos_do_dia.index = jogos_do_dia.index.set_names(['Nº'])
Jogos_do_Dia = jogos_do_dia.rename(index=lambda x: x + 1)
#Jogos_do_Dia

In [10]:
lista=[]

for a,b,c,d,e,f,g,h,i,j,k,l in zip(Jogos_do_Dia.League,Jogos_do_Dia.Date,Jogos_do_Dia.Time,Jogos_do_Dia.Home,Jogos_do_Dia.Away,
                                   Jogos_do_Dia.FT_Odd_H,Jogos_do_Dia.FT_Odd_D,Jogos_do_Dia.FT_Odd_A,Jogos_do_Dia.FT_Odd_Over25,Jogos_do_Dia.FT_Odd_Under25,
                                   Jogos_do_Dia.FT_Odd_BTTS_Yes,Jogos_do_Dia.FT_Odd_BTTS_No):
        League = a
        Date = b
        Time = c
        home = d
        away = e
        FT_Odds_H = f
        FT_Odds_D = g
        FT_Odds_A = h
        FT_Odds_Over25 = i
        FT_Odds_Under25 = j
        FT_Odds_BTTS_Yes = k
        FT_Odds_BTTS_No = l
  
        df1 = base_HBackDraw[base_HBackDraw.Home == home].tail(1)

        df2 = base_ABackDraw[base_ABackDraw.Away == away].tail(1)

        jogo = {}

        jogo["League"] = League 
        jogo["Date"] = Date
        jogo["Time"] = Time
        jogo["Home"] = home
        jogo["Away"] = away
        jogo["FT_Odds_H"] = FT_Odds_H
        jogo["FT_Odds_D"] = FT_Odds_D
        jogo["FT_Odds_A"] = FT_Odds_A
        jogo["FT_Odds_Over25"] = FT_Odds_Over25
        jogo["FT_Odds_Under25"] = FT_Odds_Under25
        jogo["FT_Odds_BTTS_Yes"] = FT_Odds_BTTS_Yes
        jogo["FT_Odds_BTTS_No"] = FT_Odds_BTTS_No

        
        try:

            base_HBackDraw = baseBackDraw[['Home', 'CV_HDA', 'Media_VG_H', 'CV_CG_H']]

            base_ABackDraw = baseBackDraw[['Away', 'CV_HDA', 'Media_VG_A', 'CV_CG_A']]
        
            
            
            
            jogo['Media_VG_H'] = df1[df1.Home == home]['Media_VG_H'].iloc[0]
            jogo['CV_CG_H'] = df1[df1.Home == home]['CV_CG_H'].iloc[0]


            jogo['Media_VG_A'] = df2[df2.Away == away]['Media_VG_A'].iloc[0]
            jogo['CV_CG_A'] = df2[df2.Away == away]['CV_CG_A'].iloc[0]
            
        except:
            pass
        
        lista.append(jogo)


df = pd.DataFrame(lista)
#df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

# CV das Odds do Match Odds
df['CV_HDA'] = df[['FT_Odds_H','FT_Odds_D','FT_Odds_A']].std(ddof=0, axis=1) / df[['FT_Odds_H','FT_Odds_D','FT_Odds_A']].mean(axis=1)
#df

In [11]:
flt = (
((df.CV_HDA >= 0.2) & (df.CV_HDA <= 0.3))
& ((df.FT_Odds_H > 1.5) & (df.FT_Odds_H < 3.5))
& ((df.FT_Odds_D > 3)) & ((df.FT_Odds_D < 4.02))
& ((df.FT_Odds_A > 1.52)) & ((df.FT_Odds_A < 4))
& ((df.FT_Odds_Under25 >= 1.01) & (df.FT_Odds_Under25 <= 3))
& ((df.Media_VG_A >= 0.153)) & ((df.Media_VG_A <= 1.85))
& ((df.CV_CG_A <= 0.8))
& ((df.CV_CG_H > 0.2)) & ((df.CV_CG_H < 0.6))
)

Back_Draw = df[flt]
Back_Draw = Back_Draw[['Date','Time','League','Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A']]
Back_Draw = drop_reset_index(Back_Draw)



from datetime import date, datetime, timedelta
data = datetime.today() + timedelta(1)
amanha = data.strftime('%Y-%m-%d')
#Back_Draw.to_excel(f'Entradas Back_Draw\\{amanha}_entrada_Back_Draw_.xlsx', index=None)
#Back_Draw

# Entradas

In [12]:
Back_Draw

,Date,Time,League,Home,Away,FT_Odds_H,FT_Odds_D,FT_Odds_A
1,09.07.2023,20:30,DIVISION PROFESIONAL - ROUND 18,Universitario de Vinto,The Strongest,3.1,3.4,2.05
